In [2]:
import pandas as pd 
import numpy as np 
from scipy.special import logsumexp

In [3]:
# Set the noise parameter sigma (standard deviation)
sigma = 1/11

data = pd.read_csv('trajectories.txt', header=None, delim_whitespace=True, encoding='utf-16')

# Get the number of rows (time steps) and columns.
T, total_cols = data.shape
M = total_cols // 2 #number f trajectories

# Extract the positions and measurements into NumPy arrays.
# positions: shape (T, M) where each column is a trajectory's true positions.
# measurements: shape (T, M) where each column is the corresponding measurement trajectory.
positions = data.iloc[:, 0::2].values  # even-indexed columns (true positions)
measurements = data.iloc[:, 1::2].values  # odd-indexed columns (measurements)

# Pre-calculate the constant part of the Gaussian log-probability.
# For a single time step, the Gaussian PDF for the measurement given the true state is:
#   P(m | x) = (1 / sqrt(2*pi*sigma^2)) * exp( - (m - x)^2 / (2*sigma^2) )
# Taking the logarithm gives:
#   ln P(m | x) = -0.5 * ln(2*pi*sigma^2) - (m - x)^2 / (2*sigma^2)
# Since the system is Markovian, the probability of the whole trajectory is the product of the
# individual probabilities. In the log domain, this becomes a sum over time steps.
log_const = -0.5 * np.log(2 * np.pi * sigma**2)

# ---------------------------------------------------
# 1. Compute log probability for each trajectory using its own positions.
# For each trajectory i, compute:
#   ln P({y^(i)} | {x^(i)}) = sum_{t=1}^{T} [ log_const - (measurement[t,i] - position[t,i])^2 / (2*sigma^2) ]
# We perform this sum over the time axis (axis 0) for each trajectory.
log_p_y_given_x_self = np.sum(log_const - ((measurements - positions)**2) / (2 * sigma**2), axis=0)
# log_p_y_given_x_self is a 1D array of shape (M,).
#element-wise log sum. 

# ---------------------------------------------------
# 2. Compute the full matrix of log probabilities.
# We want to compute, for each pair (i, j):
#   ln P({y^(i)} | {x^(j)}) = sum_{t=1}^{T} [ log_const - (measurement[t,i] - position[t,j])^2 / (2*sigma^2) ]
#
# This is the logarithm of the product over time steps, which we compute as a sum over t.
# To vectorize this, we use broadcasting.
# Create an array "diff" with shape (T, M, M) where:
#   diff[t, i, j] = measurements[t, i] - positions[t, j]
diff = measurements[:, :, np.newaxis] - positions[:, np.newaxis, :]  # shape: (T, M, M)
# Square the differences and sum over time steps (axis 0) to obtain a matrix of shape (M, M):
sum_sq_diff = np.sum(diff**2, axis=0)  # shape: (M, M)

# Now compute the log probability matrix.
# For each (i, j):
#   log_prob_matrix[i, j] = T * log_const - (sum_sq_diff[i, j]) / (2*sigma^2)
log_prob_matrix = T * log_const - sum_sq_diff / (2 * sigma**2)
# Each element (i, j) corresponds to ln P({y^(i)} | {x^(j)}),
# which is the sum of the log probabilities over all time steps.

# ---------------------------------------------------
# 3. Estimate the marginal probability for each measurement trajectory.
# For a given measurement trajectory i, the marginal probability is approximated by averaging
# the conditional probability P({y^(i)} | {x^(j)}) over all trajectories j:
#   P({y^(i)}) ≈ (1/M) * sum_{j=1}^{M} P({y^(i)} | {x^(j)})
# In the log domain, we compute:
#   ln P({y^(i)}) = logsumexp( log_prob_matrix[i, :] ) - ln(M)
log_p_y = logsumexp(log_prob_matrix, axis=1) - np.log(M)
# log_p_y is a 1D array of shape (M,).

# ---------------------------------------------------
# 4. Compute the log ratio for each trajectory.
# For each trajectory i, the log ratio is:
#   log_ratio[i] = ln P({y^(i)} | {x^(i)}) - ln P({y^(i)})
log_ratios = log_p_y_given_x_self - log_p_y

# Finally, the mutual information I_c is the average of these log ratios over all trajectories.
mean_I_c = np.mean(log_ratios)

print("Mutual Information < I_c >:", mean_I_c)


C:\Users\dcald\AppData\Local\Temp\ipykernel_17680\1084198695.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv('trajectories.txt', header=None, delim_whitespace=True, encoding='utf-16')


Mutual Information < I_c >: 6.21460809842175
